# Week 6 - Second Year Project

---

**Learning goals**
* Know how to obtain contextualized embeddings
* Understand the task of word sense disambiguation
* Perform unsupervised word sense ambiguity detection with contextualized embeddings
* Explore how words are split in subwords
* Probe language models with the [MASK] token
* Finetune contextualized embeddings for a target task

**Notebook overview**

*Lecture 11*
1. Obtaining Elmo embeddings
2. Word sense disambiguation with Elmo

*Lecture 12*

3. Subword segmentation
4. Finetune a BERT model for a downstream task

# Lecture 11: Elmo

In contextualized embeddings (i.e. Elmo-embeddings), the embedding of a word is conditioned on its context. This means that if the same surface-form occurs in a very different context, its embeddings should be substantially different. We are going to examine whether we can exploit this difference for word sense disambiguation: the task of identifying the sense (meaning) of a word in a given context. Senses can be thought of as dictionary definitions.

For more information on the task, we refer to a description on [nlpprogress.com](http://nlpprogress.com/english/word_sense_disambiguation.html) and [chapter 18 of speech and language processing](https://web.stanford.edu/~jurafsky/slp3/old_dec21/18.pdf). Note that this is not exam material (even though we will assume you know what word-sense disambiguation is).

We will use the SemCor dataset, which is annotated with gold [WordNet](https://wordnet.princeton.edu/) senses. You can query WordNet 3.1 [here](http://wordnetweb.princeton.edu/perl/webwn) for analysis. It should be noted that some senses have changed (SemCor is based on wordnet 3.0), but in most cases they should match.

## 1. Getting Elmo embeddings for target words

We are going to make use of the [ElmoForManyLangs](https://pypi.org/project/elmoformanylangs/0.0.3/) python package to derive Elmo embeddings. It can be installed with `pip3 install elmoformanylangs`.

You can download English embeddings trained with default Elmo parameters on 20-million-words data randomly sampled from Wikipedia and CommonCrawl: http://vectors.nlpl.eu/repository/11/144.zip (after unzipping, put all the files in a separate folder).

You can generate embeddings for a sentence like this:

In [ ]:
import elmoformanylangs

#elmo.en is your unzipped folder that you downloaded
elmo_model = elmoformanylangs.Embedder('elmo.en/')

example_sent = [['this', 'is', 'an', 'example']]
embeddings = elmo_model.sents2elmo(example_sent)[0]

print(embeddings.shape)

For the ambiguity detection task, you are provided with sentence triples and a target word. The task is to identify which of `sent1` and `sent2` is the sentence with a sense that differs from `sent0` (there is always one with the same sense, and one with a different sense). Consider for example the following datapoint:

In [ ]:
target_word = 'laugh'
sent0 = ['Amy', 'did', 'not', 'laugh', 'this', 'was', 'work', 'concentration', 'achievement']
sent1 = ['She', 'began', 'to', 'laugh']
sent2 = ['And', 'the', 'old_man', 'had', 'given', 'a', 'sly', 'and', 'wicked', 'laugh', 'and', 'said', 'Hell', 'yes']


In this example, the sentence that has a different sense for the target word `laugh` is `sent2`, as it uses a "noun sense" (see [laugh](http://wordnetweb.princeton.edu/perl/webwn?s=laugh&sub=Search+WordNet&o2=&o0=1&o8=1&o1=1&o7=&o5=&o9=&o6=&o3=&o4=&h=) on wordnet), whereas `sent1` uses the same sense as `sent0`. We will refer to this as gold label `2`, and when `sent1` uses a different sense, it will be gold label `1`.

We can compare the contextualized embeddings of all the `laugh` embeddings across the sentences by using cosine distance. The steps are as follows:
* a) calculate the elmo embeddings for the sentences
* b) extract the embedding of the first occurence of the target word
* c) calculate the cosine distance (from [week 4](https://github.itu.dk/robv/intro-nlp2023/blob/main/assignments/week4/week4.ipynb)) between the embedding of the target word in the first sentence to the embedding of the target word in the second sentence
* d) compare this to the distance between the embedding of the target word in the first sentence to the embedding of the target word in the third sentence
* e) if the difference is larger, the target word is more likely to have a different sense (if the representations are good). Which sentence is more likely to have a different sense in the above example?

## 2. Word sense ambiguity detection 

In the file `semcor_dev.csv` in the repo, we have a larger corpus annotated for this task. You can use the cached elmo embeddings for the target words from `semcor_dev.elmo.pickle`, which contains a list of embeddings and a list of labels in exactly the same order as `semcor_dev.csv`. The gold labels are 1 and 2, which refer to the column of the sentence that has a different sense (thus sent2 and sent3 in the previous example). The columns in the file represent: instance_idx - sent1 - sent2 - sent3 - tgt_word - gold_label - pos_tags

Note:  embeds_list in the code below contains a list of 1000 examples. Each example is a list with 3 embeddings and each embedding is a vector of length 1024

* a) Get the performance (accuracy) of the majority class baseline
* b) Use the cosine distance on each of the sentence triples, and predict whether sentence 2 or 3 uses a different sens.
* c) What is the accuracy of disambiguation with the cosine distance on this data? What can we conclude based on the difference to the majority baseline?


In [ ]:
import pickle

embeds_list, labels_list = pickle.load(open('semcor_dev.elmo.pickle', 'rb'))
# Your implementation goes here:



# Lecture 12: BERT




## 3. Subword tokenization

BERT models are trained to predict tokens that were masked with a special `[mask]` token. In this assignment you will inspect what it has learned, and whether it has certain preferences (i.e. probing). Note that you need 4gb of RAM for this assignment, otherwise you can use the HPC.

a) Load the multilingual Bert tokenizer:

In [ ]:
from transformers import AutoTokenizer

tokzr = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', use_fast=False)

Multilingual BERT was trained on the 100 most frequent languages of Wikipedia. They used smoothing, to correct inbalances in the data. However, their smoothing is relatively conservative, so high-resource languages have a higher impact on the model, and it is unclear how they sampled for training the tokenizer. Compare the tokenizations for two different language types you know; preferably one higher-resource and one lower-resource. If you only know 1 language, or only high-resource languages, try to use a different variety of the language (for example for English, use social media abbreviations or typos, e.g.: c u tmrw). Can you observe any differences in the results? does it match your intuition of separating mostly meaning-carrying subwords?

You can use Figure 1 of https://arxiv.org/pdf/1911.02116.pdf or https://en.wikipedia.org/wiki/List_of_Wikipedias to see how large languages are on Wikipedia.

In [ ]:
tokzr.tokenize('this is an example input')

b) Test whether the `bert-base-cased` model can solve the analogy task that we discussed in the word2vec lecture ([slides](https://github.itu.dk/robv/intro-nlp2023/blob/main/slides/07-vector_semantics.pdf), [assignment](https://github.itu.dk/robv/intro-nlp2023/blob/main/assignments/week4/week4.ipynb)), we can do this by masking the target word we are looking for, and let the model predict which words fit best. We can then use a prompt to discover what the language model would guess. For example, we can use the prompt "man is to king as woman is to [MASK]". Try at least two syntactic analogies, and two semantic analogies.
You can use the following code:

In [ ]:
from transformers import AutoModelForMaskedLM,AutoTokenizer
import torch

def getTopN(inputSent, model, tokzr, topn=1):
    maskId = tokzr.convert_tokens_to_ids(tokzr.mask_token)
    tokenIds = tokzr(inputSent).input_ids
    if maskId not in tokenIds:
        return 'please include ' + tokzr.mask_token + ' in your input'
    maskIndex = tokenIds.index(maskId)
    logits = model(torch.tensor([tokenIds])).logits
    return tokzr.convert_ids_to_tokens(torch.topk(logits, topn, dim=2).indices[0][maskIndex])

model = AutoModelForMaskedLM.from_pretrained('bert-base-cased')
tokzr = AutoTokenizer.from_pretrained('bert-base-cased')

getTopN('This is a [MASK] test.', model, tokzr, 5)


c) Test how robust the language model is, does it have an effect on the results of the word predictions if you include punctuations at the end of the sentence?, what about starting with a capital? and do typos have a large impact?

d) Think of some prompts that test whether the model has any gender biases, you can test this for example by using common gendered names or pronouns, swapping them and then check whether the predicted word changed.

# 4. Finetune a BERT model

We have provided code for training a BERT based classifier, which can be found in `assignments/week6/bert/bert-topic.py`. The implementation uses huggingface's transformers library (https://github.com/huggingface/transformers), and simply adds a linear layer to convert the output of the CLS token from the last layer of the masked language model to a label. 

a) Inspect the code; what should the shape of the output_scores be at the end of the forward pass?, What does this output represent?

b) Train the model on your own machine or on the HPC without a GPU (Note that this code needs ~8gb ram), how long does it take?

c) Now change the number of maximum training sentences (MAX_TRAIN_SENTS) to 500 and the batch size (BATCH_SIZE) to 32. Note that it will now take very long to train on a normal personal computer. Train the model on the HPC, make sure you reserve a GPU to speed up the training. For more information, see http://hpc.itu.dk/scheduling/templates/gpu/ (only available on ITU network/VPN). Note that the code detects automatically whether a GPU is available. Also note that the transformers library is already installed, and can be loaded with:

```
module load PyTorch/1.7.1-foss-2020b
module load Transformers/4.2.1-foss-2020a-Python-3.8.2
``` 

(which you also have to put in the job script).